In [3]:
from matplotlib import pyplot as plt
import seaborn as sns

In [6]:
import yaml 
import glob 
import pandas as pd

class LabelAnalysis():

    def __init__(self, video_name):
        self.video_name = video_name 
        self.__path = r'C:\Users\jyc13\Documents\Python Scripts\openCV_eyemovement\yolov5\runs\detect\{0}\labels'.format(video_name)
        self.__object_classes = yaml.safe_load(open('object_classes.yaml', 'r'))
        self.read_video_info()

        self.__video_w = int(self.video_info['FrameWidth']) # 가로 resolution
        self.__video_h = int(self.video_info['FrameHeight']) # 세로 resolution
        self.__video_fps = float(self.video_info['FPS']) # frame per second
        self.__start_frame = int(self.video_info['StartFrame'])
        self.__end_frame = int(self.video_info['EndFrame'])

    def read_video_info(self):
        file_path = "D:\Research\SC\DATA\Eye_Movement\VideoInfo.csv"
        self.video_info = pd.read_csv(file_path)

    def read_labels(self):
        # YOLO style: (num_class center_x center_y w, h \n) 모든 값은 resoluion 기준으로 비율
        
        # read all txt files in the __path
        files = glob.glob(f'{self.__path}/*.txt')
        data = []
        for file in files:
            n_frame = int(file.split('_')[-1].replace('.txt', ''))
            with open(file, 'rt') as f:
                lines = f.readlines()
                for line in lines:
                    splited = line.split(' ')
                    num_class = int(splited[0])
                    class_name = str(self.__object_classes[num_class])
                    # pixel 단위로 변환
                    center_x = float(splited[1]) * self.__video_w 
                    center_y = float(splited[2]) * self.__video_h 
                    w = float(splited[3]) * self.__video_w 
                    h = float(splited[4]) * self.__video_h 
                    size = w * h # size는 width * height 직사각형이라고 가정

                    frame_data = [n_frame, num_class, class_name, center_x, center_y, w, h, size]
                    data.append(frame_data)

        # format Dataframe
        df = pd.DataFrame(data)
        df.rename(columns = {0:'n_frame'}, inplace = True)
        df.rename(columns = {1:'num_class'}, inplace = True)
        df.rename(columns = {2:'class_name'}, inplace = True)
        df.rename(columns = {3:'center_x'}, inplace = True)
        df.rename(columns = {4:'center_y'}, inplace = True)
        df.rename(columns = {5:'width'}, inplace = True)
        df.rename(columns = {6:'height'}, inplace = True)
        df.rename(columns = {7:'size'}, inplace = True)

        df.sort_values(by=['n_frame', 'num_class'], inplace=True, ignore_index=True)

        # diff
        df['pupil-eye_x'] = df.groupby(by=['n_frame'])['center_x'].diff()
        df['pupil-eye_y'] = df.groupby(by=['n_frame'])['center_y'].diff()

        return df 
    
    def get_result_df(self):
        res_df = self.read_labels()

        return res_df 

In [7]:
video1_name = '60 L 20hz 5ms 5s.mp4'
mouse1 = LabelAnalysis(video_name=video1_name)

res_df = mouse1.get_result_df()
display(res_df)

,n_frame,num_class,class_name,center_x,center_y,width,height,size,pupil-eye_x,pupil-eye_y
0,1,0,eye,910.50048,500.50008,143.000064,99.000036,14157.011484,NaN,NaN
1,1,1,pupil,907.00032,506.49948,69.999936,69.000012,4829.996424,-3.50016,5.99940
2,2,0,eye,914.99904,508.49964,140.000064,99.000036,13860.011376,NaN,NaN
3,2,1,pupil,913.49952,514.00008,71.000064,68.000040,4828.007192,-1.49952,5.50044
4,3,0,eye,921.00096,515.00016,141.999936,100.000008,14199.994736,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
47959,24418,1,pupil,903.50016,497.99988,101.000064,92.000016,9292.007504,1.49952,3.50028
47960,24419,0,eye,901.49952,502.50024,144.999936,100.999980,14644.990636,NaN,NaN
47961,24419,1,pupil,902.49984,504.50040,102.999936,91.000044,9372.998708,1.00032,2.00016
47962,24420,0,eye,903.99936,510.99984,144.000000,101.999952,14687.993088,NaN,NaN


In [ ]:
import yaml 
import glob 
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

# path 
path = r'C:\Users\jyc13\Documents\Python Scripts\openCV_eyemovement\yolov5\runs\detect\exp5\labels'

# object classes
object_classes = yaml.safe_load(open('object_classes.yaml', 'r'))

# YOLO style: (num_class center_x center_y w, h \n) 모든 값은 resoluion 기준으로 비율

# read all txt files in the path
files = glob.glob(f'{path}/*.txt')
data = []
for file in files:
    n_frame = int(file.split('_')[-1].replace('.txt', ''))
    with open(file, 'rt') as f:
        lines = f.readlines()
        for line in lines:
            splited = line.split(' ')
            num_class = int(splited[0])
            class_name = str(object_classes[num_class])
            center_x = float(splited[1])
            center_y = float(splited[2])
            w = float(splited[3])
            h = float(splited[4])

            frame_data = [n_frame, num_class, class_name, center_x, center_y, w, h]
            data.append(frame_data)

# format Dataframe
df = pd.DataFrame(data)
df.rename(columns = {0:'n_frame'}, inplace = True)
df.rename(columns = {1:'num_class'}, inplace = True)
df.rename(columns = {2:'class_name'}, inplace = True)
df.rename(columns = {3:'center_x'}, inplace = True)
df.rename(columns = {4:'center_y'}, inplace = True)
df.rename(columns = {5:'width'}, inplace = True)
df.rename(columns = {6:'height'}, inplace = True)

df.sort_values(by=['n_frame', 'num_class'], inplace=True, ignore_index=True)

# diff
df['diff_x'] = df.groupby(by=['n_frame'])['center_x'].diff()
df['diff_y'] = df.groupby(by=['n_frame'])['center_y'].diff()
display(df)

In [ ]:
# x-axis
sns.lineplot(x='n_frame', y='center_x', hue='class_name', data=df)
plt.show()

In [ ]:
# y-axis
sns.lineplot(x='n_frame', y='center_y', hue='class_name', data=df)
plt.show()

In [ ]:
# diff_x
sns.lineplot(x='n_frame', y='diff_x', hue='class_name', data=df)
plt.show()

In [ ]:
# diff_y
sns.lineplot(x='n_frame', y='diff_y', hue='class_name', data=df)
plt.show()

In [8]:
from burst import * 

mouse = Burst(video_name="60 L 20hz 5ms 5s.mp4")
df_onoff = mouse.get_stim_on_frames()
display(df_onoff)

60 L 20hz 5ms 5s.mp4 is found.


,n_frame,stim
0,0,on
1,1,on
2,2,on
3,3,on
4,4,on
...,...,...
17996,17996,on
17997,17997,on
17998,17998,on
17999,17999,on
